## Outline

- [Data Load](#Load)

- [Interface](#Interface)

In [1]:
from LoaderTools.libs import LibsLoader
from AnalysisTools.analytics import AnalyticsToolkit
from AnalysisTools import visual
from matplotlib.gridspec import GridSpec


import numpy as np
import matplotlib.pyplot as plt

<a id='Load'><a/>
# Data Load

In [2]:
fname = r"E:/Data/Data_LIBS/ForHolo/wrench_map"
data_handler = LibsLoader(fname)

## 1.1 Load the Dataset

In [3]:
data_handler.load_dataset(baseline_corrected = True)

In [6]:
data_handler.normalize_to_sum()

In [9]:
data_handler.automatic_feature_extraction(sigma = 1)

Performing the SIR algorithm...
Operation Completed.


In [5]:
analytics = AnalyticsToolkit()

In [10]:
labels = analytics.clustering(model = 'kmeans', 
                               n_clusters = 4, 
                               feature_cube = data_handler.features, 
                               scaler = 'minmax',
                               random_state = 5)

<a id='Interface'><a/>

# Interface Development

In [11]:
import dash
from dash import dcc, html, Output, Input,  ctx
import plotly.graph_objects as go
import plotly.express as px
from flask import Flask

import numpy as np

from AnalysisTools.analytics import AnalyticsToolkit
from AnalysisTools import visual

In [18]:
# Inicializa a aplicação Dash
app = dash.Dash(__name__)

# Converte a matriz para um gráfico plotly
fig1 = go.Figure(data=go.Heatmap(
    z=labels,
    colorscale='Earth',
    showscale=False  
))

fig1.update_layout(
    autosize=False,
    width=350,   
    height=350,  
    margin=dict(l=0, r=10, t=40, b=40),
    font=dict(size=16),
    xaxis_title="X (mm)", yaxis_title="Y (mm)", 
    xaxis=dict( tickvals=[0, 159], ticktext=['0', '160']),
    yaxis=dict( tickvals=[0, 179], ticktext=['0', '180']),
    title="Clustering",title_x=0.5
)

# Cria o gráfico de linha usando Plotly Express
fig2 = px.line(
    x=data_handler.wavelengths, 
    y=np.mean(data_handler.dataset, axis = (0, 1)), 
    labels={"x": "Wavelength (nm)", "y": "Intensity (x10<sup>-3</sup>)"},
    title="Average Spectrum"
)
fig2.update_layout(
    autosize=False,
    width=540,   # Largura do gráfico
    height=350,  # Altura do gráfico
    margin=dict(l=0, r=20, t=40, b=40),
    font=dict(size=16),
    xaxis=dict(range=[200, 900]),  # Definir o intervalo do eixo x
    yaxis=dict( range=[0, np.max(np.mean(data_handler.dataset, axis = (0, 1)))] ),    
    title_x=0.5  
)

# Cria o gráfico da tabela periódica
table = PeriodicTable()
fig3 = table.fig

Imaging_spec=data_handler.dataset[:, :, (np.abs(data_handler.wavelengths - 345.3)).argmin()]
fig4=go.Figure(data=go.Heatmap(
    z=Imaging_spec,
    zmin=np.min(Imaging_spec),
    zmax=np.max(Imaging_spec),
    colorscale='Viridis',
    showscale=False  
))
fig4.update_layout(
    autosize=False,
    width=300,   
    height=300,  
    margin=dict(l=0, r=10, t=40, b=40),
    font=dict(size=16),
#     xaxis_title="X (mm)", yaxis_title="Y (mm)", 
    xaxis=dict( tickvals=[0, 159], ticktext=['0', '160']),
    yaxis=dict( tickvals=[0, 179], ticktext=['0', '180']),
    title="Spectral Imaging",title_x=0.5
)


# Layout da aplicação Dash com os gráficos lado a lado
app.layout = html.Div([
    html.H1(children='Spectral Analysis UI',
           style={
            'textAlign': 'center', 'color': 'white', 'backgroundColor': '#969696',  
            'padding': '10px',  # Espaçamento interno (padding) ao redor do título
            'borderRadius': '5px'  # Bordas arredondadas para um efeito estético
        }),
    html.Div([
        html.Div([
            dcc.Graph(
                id='Clustering',
                figure=fig1
            )
        ], style={'width': '36%'}),

        html.Div([
            dcc.Graph(
                id='spectrum-plot',
                figure=fig2
            )
        ], style={'width': '60%'})
    ], style={
        'display': 'flex',
        'justify-content': 'space-between',
        'border': '2px solid gray',
        'padding': '10px',
        'borderRadius': '5px',
        'margin': '10px'
    }),
    html.Div([        
        html.Button('Sample Composition', id='reset-button', n_clicks=0, 
                    style={'width': '18%', 'display': 'inline-block', 'float': 'right'}),
        html.Div(id='output', style={'width': '80%', 'display': 'inline-block'})  # Output text area

    ], style={
        'display': 'flex',
        'justify-content': 'space-between',
        'alignItems': 'center',  # Alinha o botão e o texto ao centro
#         'border': '2px solid gray',
        'padding': '10px',
        'borderRadius': '5px',
        'margin': '10px'
    }),
    
    html.Div([
        html.Div([
            dcc.Graph(id='periodic-table', figure=fig3)  # Gráfico da tabela periódica
        ], style={'width': '60%'}),  # Ocupa metade da largura

        html.Div([
            dcc.Graph(id='Spectral-Img', figure=fig4)  # Novo gráfico ao lado
        ], style={'width': '36%'})  # Ocupa metade da largura
    ], style={
        'display': 'flex',
        'justify-content': 'space-between',  # Mantém os gráficos lado a lado
        'padding': '5px'
    })
])

@app.callback(
    [Output('spectrum-plot', 'figure'), Output('Spectral-Img', 'figure')],
    Input('spectrum-plot', 'clickData')
)
def update_spectrum_plot(clickData):
    fig = go.Figure(fig2)  # Cria uma cópia da figura original
    fig1 = go.Figure(fig4)  
    if clickData is not None:
        x_value = clickData['points'][0]['x']
#         print(x_value)
        # Adiciona a linha vertical no gráfico de linha
        fig.add_shape(
            type="line",
            x0=x_value, y0=0,
            x1=x_value, y1=np.max(mean_spec),
            line=dict(color="Red", width=2)
        )
        
        wavelength_index = (np.abs(data_handler.wavelengths - x_value)).argmin()
        Imaging_spec = data_handler.dataset[:, :, wavelength_index]
        
        fig1.data[0].z=Imaging_spec
        fig1.data[0].zmin = np.min(Imaging_spec)  # Define zmin diretamente no heatmap
        fig1.data[0].zmax = np.max(Imaging_spec)
        fig1.update_layout(
            title=f"Spectral Imaging at {x_value:.1f} nm"
        )

    return fig,fig1

# Callback para detectar o clique e mostrar o valor correspondente
@app.callback(
    [Output('output', 'children'), Output('periodic-table', 'figure')],
    [Input('Clustering', 'clickData'), Input('reset-button', 'n_clicks')], prevent_initial_call=True
)
def display_click_data(clickData, n_clicks):
    
    # Se o botão de reset for clicado
    if "reset-button" == ctx.triggered_id:
        table.reset()  # Chama a função reset da tabela periódica
        return "Sample Elements!", table.fig  # Retorna a tabela atualizada

    if clickData is None:
        return "Select a cluster to know the composition.", fig3 # Retorna a tabela inicial se não houver clique
    
    # Obtém as coordenadas do clique
    point = clickData['points'][0]
    x = point['x']
    y = point['y']
    value = labels[y, x]  # Obter o valor com base nas coordenadas
    
    counts = analytics.identify_from_elements(
        spectral_cube=data_handler.dataset[labels == value], 
        wavelengths=data_handler.wavelengths,
        operation='average',
        min_intensity=0.1,
        return_counts=True
    )
    
    table.update_periodic_table(counts)
    new_fig3 = table.fig
    return f"Valor clicado: {value}", new_fig3


# Executa a aplicação
if __name__ == '__main__':
    app.run_server(debug=True)   
#     app.run_server(host="192.168.1.81", port=8050)   # http://192.168.1.81:8050


In [16]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

class PeriodicTable:
    def __init__(self):
        self.symbol = [
            ['H', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'He'],
            ['Li', 'Be', '', '', '', '', '', '', '', '', '', '', 'B', 'C', 'N', 'O', 'F', 'Ne'],
            ['Na', 'Mg', '', '', '', '', '', '', '', '', '', '', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar'],
            ['K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr'],
            ['Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe'],
            ['Cs', 'Ba', '', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn'],
            ['Fr', 'Ra', '', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Uut', 'Fl', 'Uup', 'Lv', 'Uus', 'Uuo'],
            ['', '', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', ''],
            ['', '', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', '']
        ]
        
        self.element = [
            ['Hydrogen', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Helium'],
            ['Lithium', 'Beryllium', '', '', '', '', '', '', '', '', '', '', 'Boron', 'Carbon', 'Nitrogen', 'Oxygen', 'Fluorine', 'Neon'],
            ['Sodium', 'Magnesium', '', '', '', '', '', '', '', '', '', '', 'Aluminium', 'Silicon', 'Phosphorus', 'Sulfur', 'Chlorine', 'Argon'],
            ['Potassium', 'Calcium', 'Scandium', 'Titanium', 'Vanadium', 'Chromium', 'Manganese', 'Iron', 'Cobalt', 'Nickel', 'Copper', 'Zinc', 'Gallium', 'Germanium', 'Arsenic', 'Selenium', 'Bromine', 'Krypton'],
            ['Rubidium', 'Strontium', 'Yttrium', 'Zirconium', 'Niobium', 'Molybdenum', 'Technetium', 'Ruthenium', 'Rhodium', 'Palladium', 'Silver', 'Cadmium', 'Indium', 'Tin', 'Antimony', 'Tellurium', 'Iodine', 'Xenon'],
            ['Cesium', 'Barium', '', 'Hafnium', 'Tantalum', 'Tungsten', 'Rhenium', 'Osmium', 'Iridium', 'Platinum', 'Gold', 'Mercury', 'Thallium', 'Lead', 'Bismuth', 'Polonium', 'Astatine', 'Radon'],
            ['Francium', 'Radium', '', 'Rutherfordium', 'Dubnium', 'Seaborgium', 'Bohrium', 'Hassium', 'Meitnerium', 'Darmstadtium', 'Roentgenium', 'Copernicium', 'Ununtrium', 'Ununquadium', 'Ununpentium', 'Ununhexium', 'Ununseptium', 'Ununoctium'],
            ['', '', 'Lanthanum', 'Cerium', 'Praseodymium', 'Neodymium', 'Promethium', 'Samarium', 'Europium', 'Gadolinium', 'Terbium', 'Dysprosium', 'Holmium', 'Erbium', 'Thulium', 'Ytterbium', 'Lutetium', ''],
            ['', '', 'Actinium', 'Thorium', 'Protactinium', 'Uranium', 'Neptunium', 'Plutonium', 'Americium', 'Curium', 'Berkelium', 'Californium', 'Einsteinium', 'Fermium', 'Mendelevium', 'Nobelium', 'Lawrencium', '']
        ]

        self.atomic_mass = [
            [1.00794, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4.002602],
            [6.941, 9.012182, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10.811, 12.0107, 14.0067, 15.9994, 18.9984032, 20.1797],
            [22.98976928, 24.3050, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26.9815386, 28.0855, 30.973762, 32.065, 35.453, 39.948],
            [39.0983, 40.078, 44.955912, 47.867, 50.9415, 51.9961, 54.938045, 55.845, 58.933195, 58.6934, 63.546, 65.38, 69.723, 72.64, 74.92160, 78.96, 79.904, 83.798],
            [85.4678, 87.62, 88.90585, 91.224, 92.90638, 95.96, 98, 101.07, 102.90550, 106.42, 107.8682, 112.411, 114.818, 118.710, 121.760, 127.60, 126.90447, 131.293],
            [132.9054519, 137.327, 0, 178.49, 180.94788, 183.84, 186.207, 190.23, 192.217, 195.084, 196.966569, 200.59, 204.3833, 207.2, 208.98040, 209, 210, 222],
            [223, 226, 0, 267, 268, 271, 272, 270, 276, 281, 280, 285, 284, 289, 288, 293, 'unknown', 294],
            [0, 0, 138.90547, 140.116, 140.90765, 144.242, 145, 150.36, 151.964, 157.25, 158.92535, 162.500, 164.93032, 167.259, 168.93421, 173.054, 174.9668, 0],
            [0, 0, 227, 232.03806, 231.03588, 238.02891, 237, 244, 243, 247, 247, 251, 252, 257, 258, 259, 262, 0]
        ]

        self.color = [
            [.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.],
            [.1, .2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, .7, .8, .8, .8, .9, 1.],
            [.1, .2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, .6, .7, .8, .8, .9, 1],
            [.1, .2, .3, .3, .3, .3, .3, .3, .3, .3, .3, .3, .6, .7, .8, .8, .9, 1.],
            [.1, .2, .3, .3, .3, .3, .3, .3, .3, .3, .3, .3, .6, .6, .7, .7, .9, 1.],
            [.1, .2, .4, .3, .3, .3, .3, .3, .3, .3, .3, .3, .6, .6, .6, .7, .9, 1.],
            [.1, .2, .5, .3, .3, .3, .3, .3, .3, .3, .3, .3, .6, .6, .6, .6, .9, 1.],
            [0, 0, .4, .4, .4, .4, .4, .4, .4, .4, .4, .4, .4, .4, .4, .4, .4, 0],
            [0, 0, .5, .5, .5, .5, .5, .5, .5, .5, .5, .5, .5, .5, .5, .5, .5, 0]
        ]

        self.colorscale = [
        [0.0, 'rgba(255, 255, 255, 1)'], [.1, 'rgba(255, 255, 204, 0.4)'], [.2, 'rgba(255, 255, 153, 0.4)'],
        [.3, 'rgba(204, 255, 179, 0.4)'], [.4, 'rgba(153, 255, 204, 0.4)'], [.5, 'rgba(166, 236, 230, 0.4)'],
        [.6, 'rgba(179, 217, 255, 0.4)'], [.7, 'rgba(209, 198, 255, 0.4)'], [.8, 'rgba(240, 179, 255, 0.4)'],
        [.9, 'rgba(248, 128, 202, 0.4)'], [1.0, 'rgba(255, 77, 148, 0.4)']
        ]

        
        self.fig = go.Figure()  
        
        self.hover=[]
        for x in range(len(self.symbol)):
            self.hover.append([i + '<br>' + 'Atomic Mass: ' + str(j) if i else ''
                            for i, j in zip(self.element[x], self.atomic_mass[x])])
        
        self.counts_geral = analytics.identify_from_elements(spectral_cube = data_handler.dataset,
                                            wavelengths = data_handler.wavelengths,
                                            operation = 'average',
                                            min_intensity = 0.1,
                                            return_counts = True)
        self.create_periodic_table(self.counts_geral)

    def create_periodic_table(self, elements_count):

        self.highlight_colorGeral = [[0 for _ in range(18)] for _ in range(9)]

        for i, row in enumerate(self.symbol):
            for j, elem in enumerate(row):
                if elem.strip() in elements_count:
                    if elements_count[elem.strip()] >= 3:
                        self.highlight_colorGeral[i][j] = 1
                    elif 0 < elements_count[elem.strip()] < 3:
                        self.highlight_colorGeral[i][j] = .5                        
                    else:
                        self.highlight_colorGeral[i][j] = 0.2


        # Add the base periodic table
        self.fig.add_trace(go.Heatmap(
            z=self.color[::-1],
            texttemplate="%{text}",
            textfont={"size": 12},
            colorscale=self.colorscale,
            showscale=False,
            hoverinfo='text',
#             opacity=0.2,
            #hovertext=self.hover
        ))

        # Add the highlighted elements
        self.fig.add_trace(go.Heatmap(
            z=self.highlight_colorGeral[::-1],        
            text=self.symbol[::-1],
            texttemplate="%{text}",
            textfont={"size": 10, "color": "black"},
            colorscale=[[0, 'rgba(0,0,0,0)'],[0.2, 'rgb(255,0,0)'],[0.5, 'rgb(218, 218, 50)'], [1, 'rgb(87,183,85)']],
            showscale=False,
            hoverinfo='text',
            hovertext=self.hover[::-1]
        ))

        self.fig.update_xaxes(visible=False)
        self.fig.update_yaxes(visible=False)
        self.fig.update_layout(
            title='Periodic Table with Highlighted Elements',
            title_x=0.5,
            title_font=dict(size=24),
            margin=dict(t=50, b=10, l=0, r=0),
            width=600,
            height=300
        )
#         self.fig.show()   # Tive de tirar para não aparecer repetida!!
        
    def update_periodic_table(self, elements_count):
        highlight_color = [[0 for _ in range(18)] for _ in range(9)]

        for i, row in enumerate(self.symbol):
            for j, elem in enumerate(row):
                if elem.strip() in elements_count:
                    if elements_count[elem.strip()] >= 3:
                        highlight_color[i][j] = 1
                    elif 0 < elements_count[elem.strip()] < 3:
                        highlight_color[i][j] = .5                        
                    else:
                        highlight_color[i][j] = 0.2
        
        self.fig.data[1].z = highlight_color[::-1]
#         self.fig.show()
    
    def reset(self):
        self.fig.data[1].z = self.highlight_colorGeral[::-1]
#         self.fig.show()

